---
# **Prerequisites**

**Install** dependency packages:

In [1]:
# For PDF to Image conversion
# See: https://pypi.org/project/pdf2image/
!pip install pdf2image

In [2]:
# A dependency package used by `pdf2image`
!apt-get install poppler-utils 

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 39 not upgraded.
Need to get 154 kB of archives.
After this operation, 613 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 poppler-utils amd64 0.62.0-2ubuntu2.12 [154 kB]
Fetched 154 kB in 1s (301 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 160772 files and directories currently installed.)
Preparing to unpack .../poppler-utils_0.62.0-2ubuntu2.12_amd64.deb ...
Unpacking poppler-utils (0.62.0-2ubuntu2.12) ...
Setting up poppler-utils (0.62.0-2ubuntu2.12) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [3]:
# Google Vision API
!pip install --upgrade google-cloud-vision

     |████████████████████████████████| 368kB 5.1MB/s 
     |████████████████████████████████| 51kB 5.5MB/s 


**Import** Dependencies

In [4]:
# For PDF file download
from pathlib import Path
import urllib3
# For PDF to Image conversion
from pdf2image import convert_from_path
# For non-empty directory deletion
import shutil
# Some std libs
from functools import reduce
import os
import io
# Google Cloud Vision API
from google.cloud import vision
from google.cloud.vision import AnnotateImageResponse
# JSON
import json
# Zip
from zipfile import ZipFile

---
# **Specify Budget File URL (PDF)** 

In [5]:
PDF_FILE_URLS = {
    # งบกลาง สำนักนายกฯ กลาโหม
    "2022.3.1": "https://raw.githubusercontent.com/kaogeek/thailand-budget-pdf2csv/main/budget-pdf/8.pdf",
    # คลัง ตปท. ท่องเที่ยว พัฒนาสังคมฯ
    "2022.3.2": "https://raw.githubusercontent.com/kaogeek/thailand-budget-pdf2csv/main/budget-pdf/9.pdf",
    # อุดมฯ(1)
    "2022.3.3(1)": "https://raw.githubusercontent.com/kaogeek/thailand-budget-pdf2csv/main/budget-pdf/10.pdf",
    # อุดมฯ(2)
    "2022.3.3(2)": "https://raw.githubusercontent.com/kaogeek/thailand-budget-pdf2csv/main/budget-pdf/11.pdf",
    # อุดมฯ(3)
    "2022.3.3(3)": "https://raw.githubusercontent.com/kaogeek/thailand-budget-pdf2csv/main/budget-pdf/12.pdf",
    # อุดมฯ(4)
    "2022.3.3(4)": "https://raw.githubusercontent.com/kaogeek/thailand-budget-pdf2csv/main/budget-pdf/13.pdf",
    # อุดมฯ(5)
    "2022.3.3(5)": "https://raw.githubusercontent.com/kaogeek/thailand-budget-pdf2csv/main/budget-pdf/14.pdf",
    # เกษตรฯ
    "2022.3.4": "https://raw.githubusercontent.com/kaogeek/thailand-budget-pdf2csv/main/budget-pdf/15.pdf",
    # คมนาคม ดิจิทัลฯ
    "2022.3.5": "https://raw.githubusercontent.com/kaogeek/thailand-budget-pdf2csv/main/budget-pdf/16.pdf",
    # ทรัพย์ฯ
    "2022.3.6": "https://raw.githubusercontent.com/kaogeek/thailand-budget-pdf2csv/main/budget-pdf/17.pdf",
    # พลังงาน พาณิชย์
    "2022.3.7": "https://raw.githubusercontent.com/kaogeek/thailand-budget-pdf2csv/main/budget-pdf/18.pdf",
    # มหาดไทย
    "2022.3.8": "https://raw.githubusercontent.com/kaogeek/thailand-budget-pdf2csv/main/budget-pdf/19.pdf",
    # ยุติธรรม แรงงาน วัฒนธรรม
    "2022.3.9": "https://raw.githubusercontent.com/kaogeek/thailand-budget-pdf2csv/main/budget-pdf/20.pdf",
    # ศึกษา
    "2022.3.10": "https://raw.githubusercontent.com/kaogeek/thailand-budget-pdf2csv/main/budget-pdf/21.pdf",
    # สาธารณสุข
    "2022.3.11": "https://raw.githubusercontent.com/kaogeek/thailand-budget-pdf2csv/main/budget-pdf/22.pdf",
    # อุตสาหกรรม ไม่สังกัดนายกฯ
    "2022.3.12": "https://raw.githubusercontent.com/kaogeek/thailand-budget-pdf2csv/main/budget-pdf/23.pdf",
    # จว. กลุ่มจว. (1)
    "2022.3.13(1)": "https://raw.githubusercontent.com/kaogeek/thailand-budget-pdf2csv/main/budget-pdf/24.pdf",
    # จว. กลุ่มจว. (2)
    "2022.3.13(2)": "https://raw.githubusercontent.com/kaogeek/thailand-budget-pdf2csv/main/budget-pdf/25.pdf",
    # รัฐวิสาหกิจ
    "2022.3.14": "https://raw.githubusercontent.com/kaogeek/thailand-budget-pdf2csv/main/budget-pdf/26.pdf",
    # สภา ศาล องค์กรอิสระ
    "2022.3.15": "https://raw.githubusercontent.com/kaogeek/thailand-budget-pdf2csv/main/budget-pdf/27.pdf",
    # ท้องถิ่น (1)
    "2022.3.16(1)": "https://raw.githubusercontent.com/kaogeek/thailand-budget-pdf2csv/main/budget-pdf/28.pdf",
    # ท้องถิ่น (2)
    "2022.3.16(2)": "https://raw.githubusercontent.com/kaogeek/thailand-budget-pdf2csv/main/budget-pdf/29.pdf",
    # ท้องถิ่น (3)
    "2022.3.16(3)": "https://raw.githubusercontent.com/kaogeek/thailand-budget-pdf2csv/main/budget-pdf/30.pdf",
    # หน่วยงานอื่น กาชาด ส่วนราชการในพระองค์
    "2022.3.17": "https://raw.githubusercontent.com/kaogeek/thailand-budget-pdf2csv/main/budget-pdf/31.pdf"
}

---
# **Download All PDF Files** 

In [6]:
# Clear "./pdf" dir if exists
if os.path.isdir("./pdf"):
  # Remove folder recursively
  shutil.rmtree("./pdf")
  # Make an empty dir
  os.mkdir("./pdf")
else:
  # Make dir if does not exist
  os.mkdir("./pdf")

In [7]:
http = urllib3.PoolManager()

for key in PDF_FILE_URLS.keys():
  # Save to local file path: "./pdf/<key>.pdf"
  pdfFile = Path("./pdf/"+key+".pdf")
  # Download and write to local file
  pdfFile.write_bytes(http.request("GET", PDF_FILE_URLS[key]).data)

/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is be

---
# **Convert to Images** 

In [8]:
# Create dir "./image" to store output images
if not os.path.isdir("./image"):
  os.mkdir("./image")

In [9]:
PDF_IMAGES = {}

for key in PDF_FILE_URLS.keys():
  print("Converting file: "+key)

  # Output folder path: ./image/<pdf-file-name>
  output_folder = "./image/"+key

  # Clear output dir for this key if exists
  if os.path.isdir(output_folder):
    # Remove this folder recursively
    shutil.rmtree(output_folder)
    # Make an empty dir
    os.mkdir(output_folder)
  else:
    # Make dir if does not exist
    os.mkdir(output_folder)

  # Convert pdf into images
  # Note: we've to pass "paths_only=True" to prevent OOM error
  # since the pdf file might be big (contain hundreds of pages)
  paths = convert_from_path("./pdf/"+key+".pdf", 
                            fmt="jpeg", 
                            output_folder=output_folder, 
                            output_file=key+".", 
                            paths_only=True)

  # Store image file path in dict for further use
  PDF_IMAGES[key] = paths

Converting file: 2022.3.1
Converting file: 2022.3.2
Converting file: 2022.3.3(1)
Converting file: 2022.3.3(2)
Converting file: 2022.3.3(3)
Converting file: 2022.3.3(4)
Converting file: 2022.3.3(5)
Converting file: 2022.3.4
Converting file: 2022.3.5
Converting file: 2022.3.6
Converting file: 2022.3.7
Converting file: 2022.3.8
Converting file: 2022.3.9
Converting file: 2022.3.10
Converting file: 2022.3.11
Converting file: 2022.3.12
Converting file: 2022.3.13(1)
Converting file: 2022.3.13(2)
Converting file: 2022.3.14
Converting file: 2022.3.15
Converting file: 2022.3.16(1)
Converting file: 2022.3.16(2)
Converting file: 2022.3.16(3)
Converting file: 2022.3.17


In [10]:
# Print conversion result
PDF_IMAGES

{'2022.3.1': ['./image/2022.3.1/2022.3.1.0001-001.jpg',
  './image/2022.3.1/2022.3.1.0001-002.jpg',
  './image/2022.3.1/2022.3.1.0001-003.jpg',
  './image/2022.3.1/2022.3.1.0001-004.jpg',
  './image/2022.3.1/2022.3.1.0001-005.jpg',
  './image/2022.3.1/2022.3.1.0001-006.jpg',
  './image/2022.3.1/2022.3.1.0001-007.jpg',
  './image/2022.3.1/2022.3.1.0001-008.jpg',
  './image/2022.3.1/2022.3.1.0001-009.jpg',
  './image/2022.3.1/2022.3.1.0001-010.jpg',
  './image/2022.3.1/2022.3.1.0001-011.jpg',
  './image/2022.3.1/2022.3.1.0001-012.jpg',
  './image/2022.3.1/2022.3.1.0001-013.jpg',
  './image/2022.3.1/2022.3.1.0001-014.jpg',
  './image/2022.3.1/2022.3.1.0001-015.jpg',
  './image/2022.3.1/2022.3.1.0001-016.jpg',
  './image/2022.3.1/2022.3.1.0001-017.jpg',
  './image/2022.3.1/2022.3.1.0001-018.jpg',
  './image/2022.3.1/2022.3.1.0001-019.jpg',
  './image/2022.3.1/2022.3.1.0001-020.jpg',
  './image/2022.3.1/2022.3.1.0001-021.jpg',
  './image/2022.3.1/2022.3.1.0001-022.jpg',
  './image/2022.3.1/

---
# **OCR Images to Text Files**
using **`Google Vision API`**

In [13]:
# Import `Google Cloud API` credential file

# Note: `credentials.json` must be uploaded into runtime
# before running cells below
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "credentials.json"

print("Service Account Key: {}".format(os.environ["GOOGLE_APPLICATION_CREDENTIALS"]))

vision_client = vision.ImageAnnotatorClient()

Service Account Key: credentials.json


In [14]:
# Page count (from all pdf files)
PAGE_COUNT = reduce(lambda a, b: a + b, map(lambda a: len(a), PDF_IMAGES.values()))

print("OCRing through " + str(PAGE_COUNT) + " image files ...")

OCRing through 14467 image files ...


In [15]:
def text_detection(image_path: str):
  with io.open(image_path, 'rb') as image_file:
    content = image_file.read()

  # Read image content from local file
  image = vision.Image(content=content)

  response = vision_client.text_detection(image=image)
  
  return response

In [16]:
def text_content(response) -> str:
  if response is None:
    return None

  annotations = response.text_annotations

  if annotations is not None and len(annotations) > 0:
    text = annotations[0].description
  else:
    text = None

  return text

In [17]:
def extract_text(image_path: str) -> str:
  response = text_detection(image_path)

  if response is None:
    return None

  annotations = response.text_annotations

  if annotations is not None and len(annotations) > 0:
    text = annotations[0].description
  else:
    text = None

  return text

**Extracting** images into text ucing OCR:

In [18]:
# Create dir "./text" to store output text files
if not os.path.isdir("./text"):
  os.mkdir("./text")

# Create dir "./json" to store gcloud vision 
# json response files
if not os.path.isdir("./json"):
  os.mkdir("./json")

def clear_dir(dir_path: str):
  # Clear output dir if exists
  if os.path.isdir(dir_path):
    # Remove this folder recursively
    shutil.rmtree(dir_path)
    # Make an empty dir
    os.mkdir(dir_path)
  else:
    # Make dir if does not exist
    os.mkdir(dir_path)

PDF_TEXTS = {}

for pdfDoc in PDF_IMAGES.keys():
  print()
  print("Extracting pdf doc: "+pdfDoc)
  print()

  # Output folder path: ./text/<pdf-doc>
  output_folder = "./text/"+pdfDoc
  # JSON Output folder path: ./json/<pdf-doc>
  json_output_folder = "./json/"+pdfDoc

  # Clear output dir if exists
  clear_dir(output_folder)
  clear_dir(json_output_folder)

  pages_in_txt = []

  for imageFile in PDF_IMAGES[pdfDoc]:
    print(imageFile)

    # Detect text using gcloud vision
    response = text_detection(imageFile)

    # Get text content from response
    text = text_content(response)

    # Store text content in ./text/<pdf-doc>/<image-file-name>.txt
    textFile = output_folder+"/"+imageFile[imageFile.rindex("/")+1:imageFile.rindex(".")]+".txt"
    # Store response json in ./json/<pdf-doc>/<image-file-name>.json
    jsonFile = json_output_folder+"/"+imageFile[imageFile.rindex("/")+1:imageFile.rindex(".")]+".json"

    # Write .txt
    with open(textFile, "w", encoding="utf8") as f:
      f.write(text if text is not None else "")

    # Convert gcloud response message to json with unicode decoded
    resp_json = json.loads(AnnotateImageResponse.to_json(response))

    # Write .json
    with open(jsonFile, "w", encoding="utf8") as f:
      json.dump(resp_json, f, ensure_ascii=False)

    pages_in_txt.append(text)

  PDF_TEXTS[pdfDoc] = pages_in_txt

  

Streaming output truncated to the last 5000 lines.
./image/2022.3.13(1)/2022.3.13(1).0001-070.jpg
./image/2022.3.13(1)/2022.3.13(1).0001-071.jpg
./image/2022.3.13(1)/2022.3.13(1).0001-072.jpg
./image/2022.3.13(1)/2022.3.13(1).0001-073.jpg
./image/2022.3.13(1)/2022.3.13(1).0001-074.jpg
./image/2022.3.13(1)/2022.3.13(1).0001-075.jpg
./image/2022.3.13(1)/2022.3.13(1).0001-076.jpg
./image/2022.3.13(1)/2022.3.13(1).0001-077.jpg
./image/2022.3.13(1)/2022.3.13(1).0001-078.jpg
./image/2022.3.13(1)/2022.3.13(1).0001-079.jpg
./image/2022.3.13(1)/2022.3.13(1).0001-080.jpg
./image/2022.3.13(1)/2022.3.13(1).0001-081.jpg
./image/2022.3.13(1)/2022.3.13(1).0001-082.jpg
./image/2022.3.13(1)/2022.3.13(1).0001-083.jpg
./image/2022.3.13(1)/2022.3.13(1).0001-084.jpg
./image/2022.3.13(1)/2022.3.13(1).0001-085.jpg
./image/2022.3.13(1)/2022.3.13(1).0001-086.jpg
./image/2022.3.13(1)/2022.3.13(1).0001-087.jpg
./image/2022.3.13(1)/2022.3.13(1).0001-088.jpg
./image/2022.3.13(1)/2022.3.13(1).0001-089.jpg
./image/2

In [ ]:
# Print OCR result
PDF_TEXTS

---
# **Write Combined JSON**

In [ ]:
# See: https://stackoverflow.com/questions/18337407/saving-utf-8-texts-with-json-dumps-as-utf8-not-as-u-escape-sequence
with open("budget_pdf2txt.json", "w", encoding="utf8") as file:
  json.dump(PDF_TEXTS, file, ensure_ascii=False)

---
# **Zip Output**

In [ ]:
# See: https://stackoverflow.com/questions/1855095/how-to-create-a-zip-archive-of-a-directory-in-python
def zipdir(path, ziph):
  # ziph is zipfile handle
  for root, dirs, files in os.walk(path):
    for file in files:
        ziph.write(os.path.join(root, file), 
          os.path.relpath(os.path.join(root, file), 
            os.path.join(path, '..')))

In [ ]:
# Create a ZipFile object
zipObj = ZipFile("budget_pdf2txt.zip", "w")

# Add output files/folder to the zip
zipdir("./pdf", zipObj)
zipdir("./image", zipObj)
zipdir("./text", zipObj)
zipObj.write("budget_pdf2txt.json")

# Close the Zip File
zipObj.close()

In [ ]:
# Create a ZipFile object for json
zipObj = ZipFile("budget_pdf2txt_gcloud_vision_json.zip", "w")

# Add output files/folder to the zip
zipdir("./json", zipObj)

# Close the Zip File
zipObj.close()

# **Upload Result to Google Drive**

In [ ]:
# Clear output files & folders if exists
def clear_folder(output_folder): 
  if os.path.isdir(output_folder):
    # Remove this folder recursively
    shutil.rmtree(output_folder)
    # Make an empty dir
    os.mkdir(output_folder)
  else:
    # Make dir if does not exist
    os.mkdir(output_folder)

clear_folder("./drive/MyDrive/Colab Notebooks/output/pdf2txt")

In [ ]:
# Upload Output Zip
shutil.copyfile("./budget_pdf2txt.zip", "./drive/MyDrive/Colab Notebooks/output/pdf2txt/budget_pdf2txt.zip")

In [28]:
# Upload Output JSON Zip
shutil.copyfile("./budget_pdf2txt_gcloud_vision_json.zip", "./drive/MyDrive/Colab Notebooks/output/pdf2txt/budget_pdf2txt_gcloud_vision_json.zip")

'./drive/MyDrive/Colab Notebooks/output/pdf2txt/json.zip'

In [ ]:
os.mkdir("./drive/MyDrive/Colab Notebooks/output/pdf2txt/budget_pdf2txt")

# Upload Output Files & Folders
shutil.copytree("./pdf", "./drive/MyDrive/Colab Notebooks/output/pdf2txt/budget_pdf2txt/pdf")
shutil.copytree("./image", "./drive/MyDrive/Colab Notebooks/output/pdf2txt/budget_pdf2txt/image")
shutil.copytree("./text", "./drive/MyDrive/Colab Notebooks/output/pdf2txt/budget_pdf2txt/text")
shutil.copytree("./json", "./drive/MyDrive/Colab Notebooks/output/pdf2txt/budget_pdf2txt/json")
shutil.copyfile("./budget_pdf2txt.json", "./drive/MyDrive/Colab Notebooks/output/pdf2txt/budget_pdf2txt/budget_pdf2txt.json")